In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from itertools import product
from collections import defaultdict

import json
import ast
import csv

import random

from IPython.display import display
import copy
import time

In [3]:
## To encode the probabilistic patterns
def encode_probabilistic_patterns(patterns_list, patterns_file):
    f = open(patterns_file, 'w', newline='')

    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    for pattern in patterns_list:
        values_str = [json.dumps(i) for i in pattern]
        writer.writerow(values_str)

    # close the file
    f.close()

## To decode the probabilistic patterns
def decode_probabilistic_patterns(patterns_file):
    patterns_list = []
    
    f = open(patterns_file, 'r')
    
    reader = csv.reader(f)
    
    for row in reader:
        values = [ast.literal_eval(i) for i in row]
        
        if len(values) > 0:
            patterns_list.append(values)
        
    f.close()
    
    return patterns_list

In [4]:
## Defining global variables
providers_preferences_7 = decode_probabilistic_patterns("0.5_7_probabilistic_patterns.csv")

## Defining the category_list for different levels
category_list_7 = ['E', 'Sh', 'Edu', 'Out', 'Ent', 'F', 'O']

In [5]:
def initialise_all_stuff(providers_no, microcells_no, number_of_category):
    providers_probabilistic_patterns = []
    providers_availability = []
    
    microcell_preference_dict = {}
    microcell_location_request_dict = {}
    request_sum = 0
    
    if number_of_category == 7:
        providers_preferences = providers_preferences_7
        category_list = category_list_7
    elif number_of_category == 14:
        providers_preferences = providers_preferences_14
        category_list = category_list_14
        
    for i in range(providers_no):
        providers_probabilistic_patterns.append(providers_preferences[random.randint(0, len(providers_preferences)-1)])
        providers_availability.append(random.randint(200, 500))
        
#     providers_info = sorted(zip(providers_probabilistic_patterns, providers_availability), key=lambda pair: pair[1], reverse=True)
        
    for j in range(microcells_no):
        microcell_id = j
        microcell_category = category_list[random.randint(0, len(category_list)-1)]
        microcell_request = random.randint(1000, 2000)
        request_sum += microcell_request
        
        microcell_location_request_dict[microcell_id] = {'request': microcell_request, 'category': microcell_category}
        
        if microcell_category in microcell_preference_dict:
            microcell_preference_dict[microcell_category].append(microcell_id)
        else:
            microcell_preference_dict[microcell_category] = [microcell_id]
        
    return providers_probabilistic_patterns, providers_availability, microcell_preference_dict, microcell_location_request_dict, request_sum

In [33]:
def experiment_response_rate(providers_no, microcells_no, mode, number_of_category, iterations, number_of_respondents):
    energy_usage_record_list = []
    fulfillment_ratio_record_list = []
    cost_record_list = []
    time_record_list = []
    
    for i in range(iterations):
        start_time = time.time()
        providers_probabilistic_patterns, providers_availability, microcell_preference_dict, microcell_location_request_dict, request_sum = initialise_all_stuff(providers_no, microcells_no, number_of_category)
        providers_info = sorted(zip(providers_probabilistic_patterns, providers_availability), key=lambda pair: pair[1], reverse=True)
        
        total_available_energy = np.sum(providers_availability)
        fulfillment_energy = 0
        cost = 0
        
        supplied_providers = 0
        
        respondents = random.sample(range(0, providers_no), number_of_respondents)
        
        for j in range(providers_no):
            supplying = False
            provider_probabilistic_pattern = sorted(providers_probabilistic_patterns[j][0].items(), key=lambda item: item[1], reverse=True)   
            provider_availability = providers_availability[j]
            
            if mode == 'random':
                loc_key = random.choice(list(microcell_location_request_dict))
                request = microcell_location_request_dict[loc_key]['request']
                
                if request != 0:
                    supplied_providers += 1
                    supplying = True
            elif mode == 'preference':
                loc_key = None
                probabilistic_pattern = provider_probabilistic_pattern[0]
                if probabilistic_pattern[0] in microcell_preference_dict:
                    loc_key = random.choice(microcell_preference_dict[probabilistic_pattern[0]])
                    
                if loc_key is not None:    
                    request = microcell_location_request_dict[loc_key]['request']
                    if request != 0:
                        supplied_providers += 1
                        supplying = True
                
            elif mode == 'demanding':
                loc_key = sorted(microcell_location_request_dict.items(), key=lambda item: item[1]['request'], reverse=True)[0][0]
                
                request = microcell_location_request_dict[loc_key]['request']
                
                if request != 0:
                    supplied_providers += 1
                    supplying = True
                
            elif mode == 'preference+demanding':
                provider_probabilistic_pattern = sorted(providers_info[j][0][0].items(), key=lambda item: item[1], reverse=True)
                provider_availability = providers_info[j][1]
                
                loc_key = None
                for p in provider_probabilistic_pattern:
                    preferences_locations = {}
                    if p[0] in microcell_preference_dict: # If category in microcell_preference_dict
                        for loc_id in microcell_preference_dict[p[0]]: # Add locations into the preferences_locations
                            preferences_locations[loc_id] = microcell_location_request_dict[loc_id]
                            
                        for item in sorted(preferences_locations.items(), key=lambda item: item[1]['request'], reverse=True):
                            loc_key = item[0]
                            request = microcell_location_request_dict[loc_key]['request']
                            
                            if request != 0:
                                break
                        
                        if loc_key is not None:
                            break
                            
                if loc_key is None:
                    loc_key = sorted(microcell_location_request_dict.items(), key=lambda item: item[1]['request'], reverse=True)[0][0]
                    
                request = microcell_location_request_dict[loc_key]['request']
                            
                if request != 0:
                    supplied_providers += 1
                    supplying = True
                    
            elif mode == 'demanding+preference':
                provider_probabilistic_pattern = sorted(providers_info[j][0][0].items(), key=lambda item: item[1], reverse=True)
                provider_availability = providers_info[j][1]
                
                loc_key = None
                
                ordered_demand_list = sorted(microcell_location_request_dict.items(), key=lambda item: item[1]['request'], reverse=True)
                provider_preferences = [p[0] for p in provider_probabilistic_pattern] # all provider's preference in the form of [(category, probability)]
                
                for i in ordered_demand_list:
                    if (i[1]['request'] != 0 and i[1]['category'] in provider_preferences):
                        loc_key = i[0]
                        break
                
                if loc_key is None:
                    loc_key = sorted(microcell_location_request_dict.items(), key=lambda item: item[1]['request'], reverse=True)[0][0]
                    
                request = microcell_location_request_dict[loc_key]['request']
                
                if request != 0:
                    supplied_providers += 1
                    supplying = True
                
                
            if loc_key is not None:
                loc_category = microcell_location_request_dict[loc_key]['category']

                if j in respondents:
                    if request > provider_availability:
                        fulfillment_energy += provider_availability
                        microcell_location_request_dict[loc_key]['request'] = (request - provider_availability)
                    else:
                        fulfillment_energy += request
                        microcell_location_request_dict[loc_key]['request'] = 0
                
                
            if supplying and j in respondents:
                preference_index = len(provider_probabilistic_pattern)+1
                for k in range(len(provider_probabilistic_pattern)):
                    if loc_category == provider_probabilistic_pattern[k][0]:
                        preference_index = k+1

                cost += ((provider_availability / 100) * preference_index)
        energy_usage_record_list.append(fulfillment_energy / total_available_energy)
        fulfillment_ratio_record_list.append(fulfillment_energy / request_sum)
    
        if supplied_providers != 0:
            cost_record_list.append(cost)
            
        time_record_list.append(time.time()-start_time)
    
    return np.mean(energy_usage_record_list), np.mean(fulfillment_ratio_record_list), np.mean(cost_record_list), np.mean(time_record_list)

In [34]:
def test_all_combinations_response_rate(number_of_iterations):
    level = 7
    mode = 'demanding+preference'
    variant_number = 15
    
#     variant_numbers = [5, 10, 15, 20, 25, 30]
    number_of_respondents_options = [0.2, 0.4, 0.6, 0.8, 1]
    
    result_dict_response_rate = {}
    
#     for variant_number in variant_numbers:
    for number_of_respondents_proportion in number_of_respondents_options:
        start_time = time.time()

        number_of_respondents = int(variant_number * number_of_respondents_proportion)
        print(number_of_respondents)
        EU, FR, C, T = experiment_response_rate(variant_number, 15, mode, level, number_of_iterations, number_of_respondents)

        EU_tag = '{}_{}_{}_{}_{}_EU'.format(level, variant_number, 15, number_of_respondents, mode)
        FR_tag = '{}_{}_{}_{}_{}_FR'.format(level, variant_number, 15, number_of_respondents, mode)
        C_tag = '{}_{}_{}_{}_{}_C'.format(level, variant_number, 15, number_of_respondents, mode)
        T_tag = '{}_{}_{}_{}_{}_T'.format(level, variant_number, 15, number_of_respondents, mode)
        result_dict_response_rate[EU_tag] = EU
        result_dict_response_rate[FR_tag] = FR
        result_dict_response_rate[C_tag] = C
        result_dict_response_rate[T_tag] = T

        print("Finished 1 setup")
        print("Time taken: {:.2f}".format((time.time()-start_time)/60))

    return result_dict_response_rate

In [35]:
result_dict_response_rate = test_all_combinations_response_rate(1000)

3
Finished 1 setup
Time taken: 0.00
6
Finished 1 setup
Time taken: 0.00
9
Finished 1 setup
Time taken: 0.00
12
Finished 1 setup
Time taken: 0.00
15
Finished 1 setup
Time taken: 0.01


In [36]:
result_dict_response_rate

{'7_15_15_3_demanding+preference_EU': 0.19938706338334472,
 '7_15_15_3_demanding+preference_FR': 0.046631756365358726,
 '7_15_15_3_demanding+preference_C': 17.5587,
 '7_15_15_3_demanding+preference_T': 0.0002650408744812012,
 '7_15_15_6_demanding+preference_EU': 0.401047063376461,
 '7_15_15_6_demanding+preference_FR': 0.09369003780438495,
 '7_15_15_6_demanding+preference_C': 35.98036,
 '7_15_15_6_demanding+preference_T': 0.0002629632949829102,
 '7_15_15_9_demanding+preference_EU': 0.5981686239723951,
 '7_15_15_9_demanding+preference_FR': 0.13922137260037043,
 '7_15_15_9_demanding+preference_C': 53.86397,
 '7_15_15_9_demanding+preference_T': 0.00027901458740234373,
 '7_15_15_12_demanding+preference_EU': 0.7993127448537439,
 '7_15_15_12_demanding+preference_FR': 0.18664191692775592,
 '7_15_15_12_demanding+preference_C': 72.83188999999999,
 '7_15_15_12_demanding+preference_T': 0.00029700231552124025,
 '7_15_15_15_demanding+preference_EU': 0.9974814079304457,
 '7_15_15_15_demanding+prefere

In [37]:
counter = 1

for i in result_dict_response_rate.values():
    if counter % 4 == 0:
        print(i)
    else:
        print(i ,end='\t')
    counter += 1

0.19938706338334472	0.046631756365358726	17.5587	0.0002650408744812012
0.401047063376461	0.09369003780438495	35.98036	0.0002629632949829102
0.5981686239723951	0.13922137260037043	53.86397	0.00027901458740234373
0.7993127448537439	0.18664191692775592	72.83188999999999	0.00029700231552124025
0.9974814079304457	0.23321380177954978	91.54871	0.0003730094432830811
